In [ ]:
!pip install spark-nlp==2.4.5 pyspark==2.4.4

     |████████████████████████████████| 112kB 10.0MB/s 
     |████████████████████████████████| 215.7MB 70kB/s 
     |████████████████████████████████| 204kB 41.0MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130389 sha256=ff57c96af213106706a07a50dee7a4e46474b968a37615e23f97dcc16cb87f1e
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [ ]:
!apt install openjdk-8-jdk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jdk is already the newest version (8u282-b08-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [ ]:
!update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [ ]:
!java -version

openjdk version "1.8.0_282"
OpenJDK Runtime Environment (build 1.8.0_282-8u282-b08-0ubuntu1~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.282-b08, mixed mode)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
spark = sparknlp.start() 

In [ ]:
df = spark.read.option("header", True).option("multiLine",True).option("inferSchema",True).csv("/content/drive/MyDrive/Sentence Embeddings/dataset_final_with_uid")
df1 = spark.read.option("header", True).option("multiLine",True).option("inferSchema",True).csv("/content/drive/MyDrive/Sentence Embeddings/covid_dataset_with_ids")
df2 = spark.read.option("header", True).option("multiLine",True).option("inferSchema",True).csv("/content/drive/MyDrive/Sentence Embeddings/all_combined_final")

In [ ]:
document = DocumentAssembler().setInputCol("pre_text_all_upd").setOutputCol("document")
use = UniversalSentenceEncoder.pretrained().setInputCols(["document"]).setOutputCol("USE_pre_text_all_upd_embed")

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [ ]:
df_USE = use_clf_pipeline.fit(df).transform(df)
df1_USE = use_clf_pipeline.fit(df1).transform(df1)
df2_USE = use_clf_pipeline.fit(df2).transform(df2)

In [ ]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.linalg import VectorUDT, DenseVector

ud_f = F.udf(lambda r : DenseVector(r),VectorUDT())

In [ ]:
res_df = df_USE.select("u_id",F.explode("USE_pre_text_all_upd_embed").alias("embeddings"),"mean_sent_score","final_sent_class")
res_df = res_df.select("u_id","embeddings.embeddings","mean_sent_score","final_sent_class")

In [ ]:
res_df1 = df1_USE.select("id",F.explode("USE_pre_text_all_upd_embed").alias("embeddings"),"mean_sent_score","final_sent_class")
res_df1 = res_df1.select("id","embeddings.embeddings","mean_sent_score","final_sent_class")

In [ ]:
res_df2 = df2_USE.select("u_id",F.explode("USE_pre_text_all_upd_embed").alias("embeddings"),"mean_sent_score","final_sent_class")
res_df2 = res_df2.select("u_id","embeddings.embeddings","mean_sent_score","final_sent_class")

In [ ]:
res_df = res_df.withColumn("USE_embed",ud_f(F.col("embeddings")))
res_df1 = res_df1.withColumn("USE_embed",ud_f(F.col("embeddings")))
res_df2 = res_df2.withColumn("USE_embed",ud_f(F.col("embeddings")))

In [ ]:
res_df = res_df.drop("embeddings")
res_df1 = res_df1.drop("embeddings")
res_df2 = res_df2.drop("embeddings")

In [ ]:
res_df = res_df.select("u_id","USE_embed","mean_sent_score","final_sent_class")
res_df1 = res_df1.select("id","USE_embed","mean_sent_score","final_sent_class")
res_df2 = res_df2.select("u_id","USE_embed","mean_sent_score","final_sent_class")

In [ ]:
res_df.printSchema()

root
 |-- u_id: string (nullable = true)
 |-- USE_embed: vector (nullable = true)
 |-- mean_sent_score: double (nullable = true)
 |-- final_sent_class: string (nullable = true)



In [ ]:
res_df.write.option("header",True).parquet('/content/drive/MyDrive/Sentence Embeddings/USE_EMBED_dataset_final_with_uid')
res_df1.write.option("header",True).parquet('/content/drive/MyDrive/Sentence Embeddings/USE_EMBED_covid_dataset_with_ids')
res_df2.write.option("header",True).parquet('/content/drive/MyDrive/Sentence Embeddings/USE_EMBED_all_combined_final')

In [ ]:
res_df1.cache()
res_df1.count()

In [ ]:
x ='242d96767c62edb13ccae0b7708890a8adbf6ba1d18c08e51e8f47bfe4bc6679'